In [1]:
from datasets import load_dataset

dataset = load_dataset("Kaludi/Customer-Support-Responses")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/Kaludi--Customer-Support-Responses-c8ff6fbc71b60d61/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


/opt/conda/lib/python3.10/site-packages/datasets/packaged_modules/csv/csv.py:154: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  csv_file_reader = pd.read_csv(file, iterator=True, dtype=dtype, **self.config.read_csv_kwargs)


  0%|          | 0/1 [00:00<?, ?it/s]

In [2]:
dataset

DatasetDict({
    train: Dataset({
        features: ['query', 'response'],
        num_rows: 74
    })
})

In [2]:
# Concatenate all utterances within a dialogue and map to 'dialog' key
def concatenate_utterances(example):
    example['dialog'] =  "<USER>" + example['query'] + "<ASSISTANT>" + example['response']
    return example

# Apply the function to all examples in the dataset
dataset = dataset.map(concatenate_utterances)

  0%|          | 0/74 [00:00<?, ?ex/s]

In [ ]:
#dataset['train']['dialog']

In [4]:
from transformers import AutoTokenizer , AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")

# Set the padding token to the end-of-sequence token
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained('microsoft/DialoGPT-medium')

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [6]:
encoded = tokenizer(dataset['train']['dialog'], truncation=True, padding='max_length', max_length=168, return_tensors = 'pt')

In [7]:
input_ids = encoded['input_ids']

attention_mask = encoded['attention_mask']

In [8]:
from torch.utils.data import TensorDataset

train_dataset =TensorDataset(input_ids, attention_mask)

In [9]:
from torch.utils.data import DataLoader

batch_size = 16

train_loader = DataLoader(train_dataset, batch_size = batch_size)

In [10]:
import torch.nn as nn
device_ids = [0, 1]  # IDs of the GPUs you want to use
model = model.to('cuda')
model = nn.DataParallel(model, device_ids=device_ids)

In [11]:
import torch
import torch.nn as nn
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

In [12]:
num_epochs = 50

for epoch in range(num_epochs):
    total_loss = 0.0
    model.train()
    for input_ids ,attention_mask  in train_loader:
        optimizer.zero_grad()
        input_ids = input_ids.to('cuda')
        attention_mask = attention_mask.to('cuda')
        outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)
        loss = outputs.loss
        loss.sum().backward()
        optimizer.step()
        

        total_loss += loss.sum().item()

    average_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{num_epochs} - Average Loss: {average_loss}")

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch 1/50 - Average Loss: 5.847833490371704
Epoch 2/50 - Average Loss: 2.9871429443359374
Epoch 3/50 - Average Loss: 2.6249560832977297
Epoch 4/50 - Average Loss: 2.2473045349121095
Epoch 5/50 - Average Loss: 1.9546013116836547
Epoch 6/50 - Average Loss: 1.6581624507904054
Epoch 7/50 - Average Loss: 1.3974198341369628
Epoch 8/50 - Average Loss: 1.213204550743103
Epoch 9/50 - Average Loss: 1.049602174758911
Epoch 10/50 - Average Loss: 0.9454992055892945
Epoch 11/50 - Average Loss: 0.8614281058311463
Epoch 12/50 - Average Loss: 0.8025826811790466
Epoch 13/50 - Average Loss: 0.7631384849548339
Epoch 14/50 - Average Loss: 0.7297192811965942
Epoch 15/50 - Average Loss: 0.7031514525413514
Epoch 16/50 - Average Loss: 0.6684106111526489
Epoch 17/50 - Average Loss: 0.6336804389953613
Epoch 18/50 - Average Loss: 0.6044877886772155
Epoch 19/50 - Average Loss: 0.5955815076828003
Epoch 20/50 - Average Loss: 0.5672674179077148
Epoch 21/50 - Average Loss: 0.5451953172683716
Epoch 22/50 - Average Los

In [13]:
# Save the model
model.module.save_pretrained("Model")
# Save the tokenizer
tokenizer.save_pretrained('Tokenizer')

('Tokenizer/tokenizer_config.json',
 'Tokenizer/special_tokens_map.json',
 'Tokenizer/vocab.json',
 'Tokenizer/merges.txt',
 'Tokenizer/added_tokens.json',
 'Tokenizer/tokenizer.json')

In [15]:
model = AutoModelForCausalLM.from_pretrained('/kaggle/working/Model')

tokenizer = AutoTokenizer.from_pretrained('/kaggle/working/Tokenizer')

In [27]:
while True:
    user_input =  input("You: ")
    if user_input.lower() == 'exit':
            break
    user_input = "<USER>" + user_input + "<ASSISTANT>"
    temperature = 0.7  
    max_response_length = 168  # Max bot response

    input_ids = tokenizer.encode(user_input, truncation=True, return_tensors='pt')
    with torch.no_grad():
        response = model.generate(input_ids, pad_token_id=tokenizer.eos_token_id, max_length=max_response_length, temperature=temperature)
    bot_response = tokenizer.decode(response[0], skip_special_tokens=True)
    # Remove user input from bot response
    bot_response = bot_response[len(user_input):].strip()
    print("Bot: " + bot_response)


You:  problem


Bot: We apologize for the inconvenience. Can you please provide a description of the issue you're experiencing and your email address so we can follow up with you?


You:  i am facing an issue


Bot: We apologize for the inconvenience. Can you please provide a description of the issue you're experiencing and your email address so we can follow up with you?


You:  exit
